# Knapsack

In [1]:
import random
n = 100
w_max = 250
values = [random.randint(1,10) for _ in range(n)]
weights = [random.randint(1,10) for _ in range(n)]

In [2]:
def available_actions(state):
  return [0,1]

def reward(state, action):
  if action == 0:
    return 0
  return values[state[0]]

def next_state(state, action):
  item_ind = state[0]
  if action == 0:
    return (item_ind+1, state[1])
  if action == 1:
    return (item_ind+1, state[1] - weights[item_ind])


def terminal_state(state):
  if state[1] < 0:
    return True, (-1000000, -1)
  if state[0] >= len(values):
    return True, (0, -1)
  else:
    return False, ()

cache = {}
def bellman(state):
  is_terminal, term_return = terminal_state(state)
  if is_terminal:
    return term_return
  if state in cache:
    return cache[state]
  best_value = None
  best_action = None
  for action in available_actions(state):
    action_value = reward(state, action) + bellman(next_state(state,action))[0]
    if best_value is None or action_value > best_value:
      best_value = action_value
      best_action = action
  cache[state] = (best_value, best_action)
  return best_value, best_action

In [3]:
start_state = (0, w_max)

state = start_state

actions = []
for _ in range(n):
  best_value, best_action = bellman(state)
  actions.append(best_action)
  state = next_state(state, best_action)

In [4]:
actions

[0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1]

# Egg Drop (WIP)

In [5]:
def available_actions(state):
  return range(1,state[0] + 1)

def reward(state, action):
  return -1

def next_state(state, action):
  surv_state = (state[0] - action, state[1])
  break_state = (action - 1, state[1] - 1)
  if bellman(surv_state)[0] < bellman(break_state)[0]:
    return surv_state
  else:
    return break_state

def terminal_state(state):
  if state[0] == 0:
    return True, (0, -1)
  if state[1] == 0:
    return True, (-10000000, -1)
  else:
    return False, ()

cache = {}
def bellman(state):
  is_terminal, term_return = terminal_state(state)
  if is_terminal:
    return term_return
  if state in cache:
    return cache[state]
  best_value = None
  best_action = None
  for action in available_actions(state):
    action_value = reward(state, action) + bellman(next_state(state,action))[0]
    if best_value is None or action_value > best_value:
      best_value = action_value
      best_action = action
  cache[state] = (best_value, best_action)
  return best_value, best_action

In [6]:
start_state = (100, 5)

state = start_state

actions = []
while not terminal_state(state)[0]:
  best_value, best_action = bellman(state)
  actions.append(best_action)
  state = next_state(state, best_action)